In [1]:
import pandas as pd
import numpy as np
from cryptography.hazmat.primitives import serialization
from datetime import datetime, timedelta
from src.clients import KalshiHttpClient, KalshiWebSocketClient, Environment
from src.web_scraper import get_kalshi_px, calculate_fair
from src.constants import *
from IPython.display import clear_output
import time

env = Environment.PROD # toggle environment here
KEYFILE = '../kalshi/bsun1220.txt'
KEYID = pd.read_pickle('../kalshi/key.txt')

try:
    with open(KEYFILE, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None  # Provide the password if your key is encrypted
        )
except FileNotFoundError:
    raise FileNotFoundError(f"Private key file not found at {KEYFILE}")
except Exception as e:
    raise Exception(f"Error loading private key: {str(e)}")

# Initialize the HTTP client
client = KalshiHttpClient(
    key_id=KEYID,
    private_key=private_key,
    environment=env
)

In [2]:
def get_markets(bitcoin = True):
    now = datetime.now()
    if now.hour >= DAILY_HOUR_EXPIRE:
        now = now + timedelta(days=1)
    month_dict = ['', 'JAN', 'FEB' 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    year = now.year %100
    symbol = 'BTC' if bitcoin else 'ETH'
    event_ticker = f'KX{symbol}D-{year}{month_dict[now.month]}{now.day}17'
    info = client.get(f'/trade-api/v2/events/{event_ticker}')

    tickers = [x['ticker'] for x in info['markets']]

    ans = {}
    for ticker in tickers:
        orderbook_url = f'/trade-api/v2/markets/{ticker}/orderbook'
        strike = int(np.round(float(ticker.split('-T')[1]) + 0.01, 2))
        ans[strike] = client.get(orderbook_url)['orderbook']
    
    return ans

In [3]:
trades = []

while True:
    clear_output(wait = True)
    time.sleep(TIME_SLEEP)
    for coin in ['BTC', 'ETH']:
        is_btc = coin == 'BTC'
        markets = get_markets(is_btc)
        fairs = pd.read_pickle(f'data/{coin}_fairs.pkl')
        
        for strike in fairs:
            market = markets[strike]
        
            yes_fair = int(fairs[strike] * 100)
            no_fair = 100 - yes_fair

            if market['yes'] is not None:
                for i in range(len(market['yes']) - 1, -1, -1):
                    value, size = market['yes'][i][0], market['yes'][i][1]
                    if value > yes_fair + MIN_EDGE_REQ * 100:
                        print(f'{coin}-{strike}: yes_price: {value}, yes_fair: {yes_fair}, {size} lots')

                        if BACKTEST:
                            trades.append([coin, strike, 'no', 100-value, size])
                        

            if market['no'] is not None:
                for i in range(len(market['no']) - 1, -1, -1):
                    value, size = market['no'][i][0], market['no'][i][1]
                    if value > no_fair + MIN_EDGE_REQ * 100:
                        print(f'{coin}-{strike}: no_price: {value}, no_fair: {no_fair}, {size} lots')
                        if BACKTEST:
                            trades.append([coin, strike, 'yes', 100-value, size])
            

BTC-104000: yes_price: 76, yes_fair: 72, 16 lots



KeyboardInterrupt



In [7]:
def evaluate_backtest(final_prices, trades):
    pnl = 0
    for trade in trades:
        coin, strike, cont_type, price, size = trade[0], trade[1], trade[2], trade[3], trade[4]

        if cont_type == 'yes':
            final_value = 100 if final_prices[coin] >= strike else 0
        if cont_type == 'no':
            final_value = 100 if final_prices[coin] < strike else 0

        pnl += size * (final_value - price)

    return pnl/100
        

In [13]:
final_prices = {'ETH':3300, 'BTC':95_000}
trades = [['ETH', 3200, 'yes', 90, 10], ['BTC', 96_000, 'yes', 40, 100]]
evaluate_backtest(final_prices, trades)

-39.0